# K-Nearest Neighbours

Se traduce como los k vecinos más cercanos
Clasifica cada punto en una categoría basándose en la categoría de sus vecinos más cercanos
- Es uno de los algoritmos más intuitivo y fácil de comunicar, y muestra muy buenos resultados en diversas situaciones.

**Consejo:** Cuando ajustemos un modelo KNN, es recomendable escoger una 'k' que favorezca los desempates.
- Si estamos clasificando entre vuelos que se retrasan mucho y vuelos que no, y no sabemos cómo clasificar un vuelo concreto, si solo miramos los dos vuelos más próximos en nuestro espacio de variables y uno de ellos se ha retrasado y el otro no, ¿cómo clasificar los vuelos? Para evitar este tipo de situaciones se recomienda usar un número impar, lo que va a asegurarnos que si estamos clasificando binariamente no tengamos empates.

In [1]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import pandas as pd

df = pd.read_csv(r'./base_datos_2008.csv',nrows = 1e6) # Importar el primer millón de filas

In [2]:
# Crear un nuevo dataset con estas columnas y eliminar los datos faltantes
newdf = df[['AirTime','Distance','TaxiOut','ArrDelay']].dropna()
# Generar un objeto con todas aquellas columnas que no son nuestras variables respuesta, que es el retras ode llegada
cols = newdf[newdf.columns[newdf.columns != 'ArrDelay']]
# Este código es útil si queremos ir ampliando las variables que vamos a usar para predecir esta, ya que no neceistamos modificar el objeto

In [3]:
# Crear una variable respuesta binaria
# Queremos separar entre aquellos vuelos que se hayan retrasado más de 10 minutos y aquellos que menos
filtro = newdf['ArrDelay'] > 10

In [4]:
# Hacer los datos más atractivos (no es necesario)
# Poner etiquetas a los casos
newdf['ArrDelay'][filtro] = 'Delayed'
newdf['ArrDelay'][filtro == False] = 'Not Delayed'

In [5]:
# Ver cuál es la variable respuesta
newdf['ArrDelay'].head()
# Muestra los 5 primeros casos en función de si se ha retrasado o no se ha retrasado

0    Not Delayed
1    Not Delayed
2        Delayed
3    Not Delayed
4        Delayed
Name: ArrDelay, dtype: object

### Ajustar un modelo KNN

**n_neighbors:** Indicar los números de vecinos que queremos

**n_jobs=-1:** Usar todos los núcleos del procesador

In [6]:
nbrs_3 = KNeighborsClassifier(n_neighbors = 3, n_jobs = -1)

In [7]:
# Usar las variables generadas para producir nuestra columna respuesta (ArrDelay)
nbrs_3.fit(cols,newdf['ArrDelay'])

KNeighborsClassifier(n_jobs=-1, n_neighbors=3)

In [8]:
# Generar las predicciones que podemos obtener
predicciones_3 = nbrs_3.predict(cols)

In [9]:
# Generar una columna que nos permitirá ver cuántos vuelos hemos clasificado correctamente usando los tres vecinos más cercanos
np.mean(predicciones_3 == newdf['ArrDelay'])
# Esto indica que se ha clasificado correctamente el 80.4% de los casos
# Esta medida puede ser engañosa ya que no sabemos la proporción de los datos

0.8040059625362947

In [10]:
# Ver la proporción de estos casos en nuestra base de datos
np.mean(newdf['ArrDelay'] == 'Not Delayed')
# Casi el 70% de los vuelos no se han retrasado, esto hace que nuestro modelo nos esté incrementando la predicción aproximadamente un 10% en relación a lo que pudiese predecir con solo la media

0.6912450368307814

### Correr el modelo con solo el vecino más cercano
Es posible cambiar el número de vecinos para saber cuál es el que se ajusta mejor al modelo

In [11]:
nbrs_1 = KNeighborsClassifier(n_neighbors=1,n_jobs=-1)
nbrs_1.fit(cols,newdf['ArrDelay'])
predicciones_1 = nbrs_1.predict(cols)
np.mean(predicciones_1 == newdf['ArrDelay'])

# Se observa que mirando únicamente el vecino más cercano obtiene mejores resultados que mirando los tres más cercanos, ha incrementado un 3% nuestra precisión

0.8346359548396921

In [13]:
# Comprobar
np.mean(newdf['ArrDelay'] == 'Not Delayed')

0.6912450368307814

### Explorar la matriz de confusión para ver qué tan bien se ha predicho cada caso

In [14]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(newdf['ArrDelay'],predicciones_1)
print(confusion_matrix)

# Las filas son los valores reales que tomaba la variable
# Es decir, los que se retrasaban y los que no se retrasaban
# Por columnas se ve la predicción, de los que se retrasan y los que no se retrasan

# El número mayor es el número de vuelos que no se retrasan y que hemos predicho que no se retrasan, 218051 es la cantidad de vuelos que se retrasan y que predecimos que se retrasan
# 79627 son los vuelos que no se retrasaban y que nuestro modelo predijo que se retrasaban
# 80009 son los vuelos que se retrasaban pero que nuestro modelo predijo lo contrario

[[218051  80009]
 [ 79627 587674]]
